In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib as plt
import torch
from tqdm import tqdm, trange

vertices = 1000
adjacencies = 100
pi_vals = [0.25, 0.5, 0.75]
num_trials = 1000
replicate = {}
device = torch.device("cpu")

# First Degree Neighbor

In [ ]:
######## Equation 3 ######
vertices = 1000
adjacencies = 100
pi_vals = [0.25, 0.5, 0.75]
num_trials = 1000

for pi in pi_vals:
  ADE = []
  AIE = []
  for i in range(10):
    G = nx.random_regular_graph(adjacencies, vertices)
    A = nx.adjacency_matrix(G).todense()
    W = np.random.binomial(1, pi, vertices)
    noise = np.random.standard_normal(vertices)
    print(W.shape)

    #Y = 1 - (1 - (W @ A)/ 100)**2 *(1 - W * (1/2)) + noise
    Y = W * (((W @ A)/ 100) - 3 * ((W @ A)/ 100 - 1/2)**3) + noise

    print(Y.mean())

    # this is same as before:
    average_direct_effect = (np.dot(Y,W))/sum(W) - (np.dot(Y, (1-W)))/sum((1-W))
    ADE.append(average_direct_effect)
    average_indirect_effect = ((np.dot(A @ Y, W))/sum(W) - (np.dot(A @ Y, (1-W)))/sum(1-W))/adjacencies
    AIE.append(average_indirect_effect)

  str1 = str(pi)+" ADE"
  str2 = str(pi)+" AIE"
  replicate[str1] = ADE
  replicate[str2] = AIE

# Second Degree Neighbor

In [ ]:
vertices = 1000
adjacencies = 100
pi_vals = [0.25, 0.5, 0.75]
num_trials = 1000

generate = {}

for pi in pi_vals:
  ADE = []
  AIE = []
  ASDE = []

  for i in trange(1000):
    # generate graph
    G = nx.random_regular_graph(adjacencies, vertices)

    # compute adjacencies
    A = nx.adjacency_matrix(G).todense()
    A_gpu = torch.tensor(A).float().to(device)
    A_2 = torch.matmul(A_gpu, A_gpu).cpu().numpy()
    A_2 = np.where(A_2 > 0, 1, 0)
    np.fill_diagonal(A_2, 0)
    A_2 = A_2 - A # second degree neighbors

    # simulate weights
    W = np.random.binomial(1, pi, vertices)
    noise = np.random.normal(0, 1, vertices)

    # compute Y's
    ei = W @ A_2 / np.sum(A_2, axis=1)
    Y = W * (ei - 3 * (ei - 1/2)**3) + noise

    average_direct_effect = (np.dot(Y,W))/sum(W) - (np.dot(Y, (1-W)))/sum((1-W))
    ADE.append(average_direct_effect)
    average_indirect_effect = ((np.dot(A @ Y, W))/sum(W) - (np.dot(A @ Y, (1-W)))/sum(1-W))/adjacencies
    avg_second_deg = ((np.dot(A_2 @ Y, W))/sum(W) - (np.dot(A_2 @ Y, (1-W)))/sum(1-W))/(vertices - adjacencies - 1)
    ASDE.append(avg_second_deg)
    AIE.append(average_indirect_effect)

  str1 = str(pi)+" ADE"
  str2 = str(pi)+" AIE"
  str3 = str(pi)+" ASDE"
  generate[str1] = ADE
  generate[str2] = AIE
  generate[str3] = ASDE

100%|██████████| 1000/1000 [09:08<00:00,  1.82it/s]
